# §0 Loading basic modules, data, demonstrating information gain computations

You can safely skip to [§1](#§1-Growing-Decision-Tree) where we proceed to step through growing the tree. Here I am just importing the data, giving example uses of my information gain related methods, and initializing the DTL pipeline I will use in §1.

In [39]:
import numpy as np
import pandas as pd
import utils.DTL_methods 

In [40]:
def load(): 
    """
    10 features with 12 observations
    hard coded from problem statement
    loaded in with features as rows, then transformed with features as columns
    target is last column
    """

    ### LOAD DATA
    Alt = [ "YES", "YES", "NO", "YES", "YES", "NO", "NO", "NO", "NO", "YES", "NO", "YES" ]
    Bar = ["NO", "NO", "YES", "NO", "NO", "YES", "YES", "NO", "YES", "YES", "NO", "YES"]
    Fri = ["NO", "NO", "NO", "YES", "YES", "NO", "NO", "NO", "YES", "YES", "NO", "YES"]
    # Hun = [] # excluded per problem statement
    Pat = ["Some", "Full", "Some", "Full", "Full", "Some", "None", "Some", "Full", "Full", "None", "Full",]
    Price = ["$$$","$","$","$","$$$","$$","$","$$","$","$$$","$","$"]
    Rain = ["NO", "NO", "NO", "YES", "NO", "YES", "YES", "YES", "YES", "NO", "NO", "NO"]
    Res = ["YES", "NO", "NO", "NO", "YES", "YES", "NO", "YES", "NO", "YES", "NO", "NO"]
    Type = ["French", "Thai", "Burger", "Thai", "French", "Italian", "Burger", "Thai", "Burger", "Italian", "Thai", "Burger"]
    Est = ["0-10", "30-60", "0-10" , "10-30", ">60", "0-10", "0-10", "0-10", ">60", "10-30", "0-10", "30-60"]
    # target
    Target = ["Y","N","Y","Y","N","Y","N","Y","N",'N',"N","Y"] #WillWait

    Vars = [Alt, Bar, Fri, Pat, Price, Rain, Res, Type, Est, Target]


    ### ENCODER DICTIONARY
    encoder_dict = {"NO":0, 
                         "YES":1,
                         "N":0, 
                         "Y":1,
                         "None":0, 
                         "Some":1, 
                         "Full":2, 
                         "$":0, 
                         "$$":1, 
                         "$$$":2, 
                         "French":0, 
                         "Thai":1, 
                         "Burger":2, 
                         "Italian":3, 
                         "0-10":0, 
                         "10-30":1, 
                         "30-60":2, 
                         ">60":3}
    

    ### TRANSFORM DATA [ENCODE]
    transformed_vars = []
    for Var in Vars:
        transformed_vars.append([encoder_dict[x] for x in Var])

    X = np.array([x for x in transformed_vars])
    #print(X.shape) # features loaded in rows

    df = pd.DataFrame(X.T) # cols are features
    print("%s observations consisting of %s features, including target" % (df.shape))
    
    return df

df = load()
df.head()

12 observations consisting of 10 features, including target


,0,1,2,3,4,5,6,7,8,9
0,1,0,0,1,2,0,1,0,0,1
1,1,0,0,2,0,0,0,1,2,0
2,0,1,0,1,0,0,0,2,0,1
3,1,0,1,2,0,1,0,1,1,1
4,1,0,1,2,2,0,1,0,3,0


In [41]:
### Feature Names

feature_names = ["Alt", "Bar", "Fri", "Pat", "Price", "Rain", "Res", "Type", "Est"]

In [42]:
### METHODS DEMONSTRATION: 
# entropy, information_gain, compute_info_purity from "utils.DTL_methods" package
#
# entropy –– used for computing information gain
# information_gain –– computes information gain
# compute_info_purity –– used to compute information purity from a given target vector

# entropy & information gain
num_features = df.shape[1]-1
for i in range(num_features):
    entropy = utils.DTL_methods.entropy(df, attribute=i, debug_flag=False)
    info_gain = utils.DTL_methods.info_gain(dataset=df, attribute=i, debug_flag=False)
    print('splitting on feature: %s, results in entropy: %s and info gain: %s' % (i,round(entropy,4), round(info_gain,4)))
    

splitting on feature: 0, results in entropy: 1.0 and info gain: 0.0
splitting on feature: 1, results in entropy: 1.0 and info gain: 0.0
splitting on feature: 2, results in entropy: 0.9793 and info gain: 0.0207
splitting on feature: 3, results in entropy: 0.4591 and info gain: 0.5409
splitting on feature: 4, results in entropy: 0.8043 and info gain: 0.1957
splitting on feature: 5, results in entropy: 0.9793 and info gain: 0.0207
splitting on feature: 6, results in entropy: 0.9793 and info gain: 0.0207
splitting on feature: 7, results in entropy: 1.0 and info gain: 0.0
splitting on feature: 8, results in entropy: 0.7925 and info gain: 0.2075


In [43]:
cache[0][0][0]

3

In [44]:
### DTL pipeline

def split(dataset, cache=[]):
    """
    dataset input, 
    computes the attribute to split on, adds attribute to cache,
    outputs list of resulting sub-datasets 
    (with list label indicating what value the attribute attains on that sub-dataset)
    """
    
    ### compute splitting attribute which maximizes info gain
    num_features = dataset.shape[1] - 1 # not counting target
    gains = []
    for i in range(num_features):
        gains.append(utils.DTL_methods.info_gain(dataset,i))

#     split_on = gains.index(max(gains)) # here we don't distinguish between if multiple features tie for best info gain

    splitting_attributes = [i for i, j in enumerate(gains) if j == max(gains)]
    split_on = splitting_attributes[0]
    ### compute subdatasets
    values = set(dataset[split_on])
    results = []
    
    for x in values:
        results.append([x,utils.DTL_methods.subset(dataset,split_on,x)])
    
    cache.append([[split_on,splitting_attributes,max(gains)], results])
    
    return results, cache
                    

# example
results, cache = split(df, cache=[])
print("split on attribute %s" % cache[0][0][0], '\n'+'%'*40)

for pair in results:
    print("attribute %s = %s: \nsub-dataset" % (cache[0][0][0],pair[0]))
    print(pair[1])

split on attribute 3 
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
attribute 3 = 0: 
sub-dataset
    0  1  2  3  4  5  6  7  8  9
6   0  1  0  0  0  1  0  2  0  0
10  0  0  0  0  0  0  0  1  0  0
attribute 3 = 1: 
sub-dataset
   0  1  2  3  4  5  6  7  8  9
0  1  0  0  1  2  0  1  0  0  1
2  0  1  0  1  0  0  0  2  0  1
5  0  1  0  1  1  1  1  3  0  1
7  0  0  0  1  1  1  1  1  0  1
attribute 3 = 2: 
sub-dataset
    0  1  2  3  4  5  6  7  8  9
1   1  0  0  2  0  0  0  1  2  0
3   1  0  1  2  0  1  0  1  1  1
4   1  0  1  2  2  0  1  0  3  0
8   0  1  1  2  0  1  0  2  3  0
9   1  1  1  2  2  0  1  3  1  0
11  1  1  1  2  0  0  0  2  2  1


# §1 Growing Decision Tree

We step through the process of growing a decision tree from the problem dataset, giving intermediate results and calculations at each step, and showing how the decision tree is split (computation of information gain is automated via the information gain utility method). Reading through, I hope it is fairly obvious how the entire process, including printing to console or otherwise of intermediate results, could be automated in an iterative fashion (including potentially a stopping / pruning heuristic to prevent tree overfitting).

### step 1

In [45]:
current_iteration = 0

results, cache = split(df, cache=[])

splitting_info = cache[0][0]
attrib_name = feature_names[splitting_info[0]]
print("Split on attribute '%s', with information gain %s" % (attrib_name, splitting_info[-1]))
if len(splitting_info[1])>1:
    print(", ".join([feature_names[x] for x in splitting_info[1]])+" tied for optimal information gain \n(tie breaking was carried out arbitrarily)")
print('%'*50+'\n'*2)


pat_vals = ["None","Some","Full"]
target_vals = ["No","Yes"]

for pair in results:
    attrib_val = pair[0]
    subdataset = pair[1]
    target = subdataset.iloc[:,-1]
    impurity = utils.DTL_methods.compute_info_purity(target)
    print("Splitting on %s = '%s' results in impurity of %s " % (attrib_name, pat_vals[attrib_val], impurity))
    if impurity == 0:
        label = list(target)[0]
        print("Resulting Class Label: %s (i.e., %s)" % (label, target_vals[label]), end='\n'*2) 

Split on attribute 'Pat', with information gain 0.5408520829727552
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


Splitting on Pat = 'None' results in impurity of 0 
Resulting Class Label: 0 (i.e., No)

Splitting on Pat = 'Some' results in impurity of 0 
Resulting Class Label: 1 (i.e., Yes)

Splitting on Pat = 'Full' results in impurity of 0.9182958340544896 


### Interpretation: 
Splitting along the feature Pat = 0,1 the impurity is now 0, i.e., the target lies entirely in one class on the resulting data subsets and hence those splits result in leaf nodes. For Pat = 2, we must continue to grow the DT from this node. So we arrive at an initial decision tree:
![decision tree at step 1](trees/tree1.png "tree 1")
### step 2

In [46]:
current_iteration = 1

previous_results = cache[current_iteration-1][1]
new_df = previous_results[2][1]

## decided not to bother stripping out used attributes 
## this is code that would have adjusted namespace indexing if I did this
# removed_feature = cache[-1][0][0]
# remaining_features = [x for x in feature_names if x != remaining_features[removed_feature]]
# remaining_features = feature_names

results, cache = split(new_df, cache)

splitting_info = cache[-1][0]
attrib_name = feature_names[splitting_info[0]]
print("Split on attribute '%s', with information gain %s" % (attrib_name, splitting_info[-1]))

if len(splitting_info[1])>1:
    print(", ".join([feature_names[x] for x in splitting_info[1]])+" tied for optimal information gain \n(tie breaking was carried out arbitrarily)")
print('%'*50+'\n'*2)

poss_att_vals = ["$","$$","$$$"]

for pair in results:
    attrib_val = pair[0]
    subdataset = pair[1]
    target = subdataset.iloc[:,-1]
    impurity = utils.DTL_methods.compute_info_purity(target)
    print("Splitting on %s = '%s' results in impurity of %s " % (attrib_name, poss_att_vals[attrib_val], impurity))
    if impurity == 0:
        label = list(target)[0]
        print("Resulting Class Label: %s (i.e., %s)" % (label, target_vals[label]), end='\n'*2) 

Split on attribute 'Price', with information gain 0.2516291673878229
Price, Res, Type, Est tied for optimal information gain 
(tie breaking was carried out arbitrarily)
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


Splitting on Price = '$' results in impurity of 1.0 
Splitting on Price = '$$$' results in impurity of 0 
Resulting Class Label: 0 (i.e., No)



### Interpretation: 
    Subsequently we select the feature Price to split along. This purifies a single leaf node, resulting in a class label of No. We then split along the remaining data (cheap restaurants, i.e. Price = $), we must continue to grow the DT from this node. We grow our decision tree:
![decision tree at step 2](trees/tree2.png "tree 2")
### step 3

In [47]:
current_iteration = 2
index_of_subdataset = 0
previous_results = cache[current_iteration-1][1]
new_df = previous_results[index_of_subdataset][1]

results, cache = split(new_df, cache)

splitting_info = cache[-1][0]
attrib_name = feature_names[splitting_info[0]]
print("Split on attribute '%s', with information gain %s" % (attrib_name, splitting_info[-1]))

if len(splitting_info[1])>1:
    print(", ".join([feature_names[x] for x in splitting_info[1]])+" tied for optimal information gain \n(tie breaking was carried out arbitrarily)")
print('%'*50+'\n'*2)

poss_att_vals = ["0-10","10-30","30-60",">60"]

for pair in results:
    attrib_val = pair[0]
    subdataset = pair[1]
    target = subdataset.iloc[:,-1]
    impurity = utils.DTL_methods.compute_info_purity(target)
    print("Splitting on %s = '%s' results in impurity of %s " % (attrib_name, poss_att_vals[attrib_val], impurity))
    if impurity == 0:
        label = list(target)[0]
        print("Resulting Class Label: %s (i.e., %s)" % (label, target_vals[label]), end='\n'*2) 

Split on attribute 'Est', with information gain 0.5
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


Splitting on Est = '10-30' results in impurity of 0 
Resulting Class Label: 1 (i.e., Yes)

Splitting on Est = '30-60' results in impurity of 1.0 
Splitting on Est = '>60' results in impurity of 0 
Resulting Class Label: 0 (i.e., No)



### Interpretation: 
    Information gain next selects 'Est' (estimated wait time) to split along. This yields completely determined rules (leaf nodes) when Est = '10-30', '>60', but an impure sub-dataset for Est = '30-60' –– we continue to grow the DT from this node:
![decision tree at step 3](trees/tree3.png "tree 3")
### step 4

In [48]:
current_iteration = 3
index_of_subdataset = 1
previous_results = cache[current_iteration-1][1]
new_df = previous_results[index_of_subdataset][1]

results, cache = split(new_df, cache)

splitting_info = cache[-1][0]
attrib_name = feature_names[splitting_info[0]]
print("Split on attribute '%s', with information gain %s" % (attrib_name, splitting_info[-1]))

if len(splitting_info[1])>1:
    print(", ".join([feature_names[x] for x in splitting_info[1]])+" tied for optimal information gain \n(tie breaking was carried out arbitrarily)")
print('%'*50+'\n'*2)

poss_att_vals = ["No","Yes"]

for pair in results:
    attrib_val = pair[0]
    subdataset = pair[1]
    target = subdataset.iloc[:,-1]
    impurity = utils.DTL_methods.compute_info_purity(target)
    print("Splitting on %s = '%s' results in impurity of %s " % (attrib_name, poss_att_vals[attrib_val], impurity))
    if impurity == 0:
        label = list(target)[0]
        print("Resulting Class Label: %s (i.e., %s)" % (label, target_vals[label]), end='\n'*2) 

Split on attribute 'Bar', with information gain 1.0
Bar, Fri, Type tied for optimal information gain 
(tie breaking was carried out arbitrarily)
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


Splitting on Bar = 'No' results in impurity of 0 
Resulting Class Label: 0 (i.e., No)

Splitting on Bar = 'Yes' results in impurity of 0 
Resulting Class Label: 1 (i.e., Yes)



### Interpretation: 
    The decision tree has completely purified the data at this step, and we halt the learning process with a complete chain of rules:
![decision tree at step 4](trees/tree4.png "tree 4")